In [26]:
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv     = nn.Conv2d(1, 8, 3)
        self.flatten  = nn.Flatten()          # NEW
        self.fc       = nn.Linear(8*26*26, 10)

    def forward(self, x):
        print(x.shape)                # NEW
        x = torch.relu(self.conv(x))
        x = self.flatten(x)                   # replaces .view(...)
        return self.fc(x)

net = Net().train()
loader = torch.utils.data.DataLoader(
    datasets.MNIST(".", train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=128, shuffle=True)
opt = optim.Adam(net.parameters(), lr=1e-3)

for epoch in range(2):                 # two quick epochs
    for x, y in loader:
        opt.zero_grad()
        loss = nn.CrossEntropyLoss()(net(x), y)
        loss.backward(); opt.step()
torch.save(net.state_dict(), "mnist.pt")


torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([12

In [9]:
net = Net()
net.load_state_dict(torch.load("mnist.pt"))
net.eval()                               # imperative for fx-tracing


Net(
  (conv): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=5408, out_features=10, bias=True)
)

In [16]:
import hls4ml, numpy as np, torch

dummy = torch.randn(1, 1, 28, 28)
cfg = hls4ml.utils.config_from_pytorch_model(
        net, input_shape=(1, 28, 28), granularity="name")



cfg['Model']['Precision']   = 'ap_fixed<16,6>'
cfg['Model']['ReuseFactor'] = 1                    
# tighten per-layer precision
for name in cfg['LayerName']:
    if 'conv' in name:
        cfg['LayerName'][name]['Precision']['weight'] = 'ap_fixed<8,3>'
    if 'fc' in name:
        cfg['LayerName'][name]['Precision']['weight'] = 'ap_fixed<8,3>'


In [20]:
hls_model = hls4ml.converters.convert_from_pytorch_model(
    net, hls_config=cfg, backend='Vitis',        # ← not 'Vivado'
    output_dir='mnist_hls', project_name='mnist')

hls_model.build(csim=True, synth=False)           # now calls vitis_hls


Interpreting Model ...
Topology:
Layer name: conv, layer type: Conv2D, input shape: [[None, 1, 28, 28]]
Layer name: relu, layer type: Activation, input shape: [[None, 8, 26, 26]]
Layer name: flatten, layer type: Reshape, input shape: [[None, 8, 26, 26]]
Layer name: fc, layer type: Dense, input shape: [[None, 5408]]
Creating HLS model

****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2022.2 (64-bit)
  **** SW Build 3670227 on Oct 13 2022
  **** IP Build 3669848 on Fri Oct 14 08:30:02 MDT 2022
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vitis_HLS/2022.2/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vitis_HLS/2022.2/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'gkapakos' on host 'gkapakos-Type1ProductConfigId' (Linux_x86_64 version 6.8.0-62-generic) on Sat Jun 28 20:26:29 EEST 2025
INFO: [HLS 200-10] On os Ubuntu 24.04.2 LTS
INFO: [HLS 200-10] In directory '/home/gkapakos/Desktop/ECE/10th_

{}

In [23]:
# after convert_from_pytorch_model(...)
hls_model.write()       # 1. generate project files
hls_model.compile()     # 2. compile & link C-simulation

# now it works
fixed = hls_model.predict(x.numpy())


Writing HLS project
Done
Writing HLS project
Done


In [24]:
testset = datasets.MNIST(".", train=False,
                         transform=transforms.ToTensor())
x, y    = next(iter(torch.utils.data.DataLoader(testset, batch_size=1024)))
with torch.no_grad(): fp32 = net(x).softmax(-1)


In [25]:
fixed = hls_model.predict(x.numpy())         # (N,10) np.float32
mse   = np.mean((fp32.numpy() - fixed)**2)
acc   = np.mean(fp32.argmax(1).numpy() == fixed.argmax(1))
print("MSE:", mse, "│ accuracy:", acc)

MSE: 185.29182 │ accuracy: 0.955078125
